In [21]:
#Dependencies
import pandas as pd
import numpy as np
import os

In [22]:
#Source data files
school_data = os.path.join('.\Resources','schools_complete.csv')
student_data = os.path.join('.\Resources','students_complete.csv')


In [23]:
#Read in csv data
load_school = pd.read_csv(school_data)
load_student = pd.read_csv(student_data)

In [24]:
#Quick summary stats of school data set
school_describe = load_school.describe()
school_describe

,School ID,size,budget
count,15.000000,15.000000,1.500000e+01
mean,7.000000,2611.333333,1.643295e+06
std,4.472136,1420.915282,9.347763e+05
min,0.000000,427.000000,2.480870e+05
25%,3.500000,1698.000000,1.046265e+06
50%,7.000000,2283.000000,1.319574e+06
75%,10.500000,3474.000000,2.228999e+06
max,14.000000,4976.000000,3.124928e+06


In [25]:
#Total Schools
total_schools = int(school_describe.iloc[0,0])
#Total District Budget
total_budget = load_school['budget'].sum()


In [26]:
student_describe = load_student.describe()
student_describe

,Student ID,reading_score,math_score
count,39170.000000,39170.00000,39170.000000
mean,19584.500000,81.87784,78.985371
std,11307.549359,10.23958,12.309968
min,0.000000,63.00000,55.000000
25%,9792.250000,73.00000,69.000000
50%,19584.500000,82.00000,79.000000
75%,29376.750000,91.00000,89.000000
max,39169.000000,99.00000,99.000000


In [27]:
#Total number of students
total_students = int(student_describe.iloc[0,0])
#Average reading score
avg_reading = round(student_describe.iloc[1,1],1)
#Average math score
avg_math = round(student_describe.iloc[1,2],1)


In [127]:
#Percent passing reading
prcnt_pass_read = round((load_student.loc[load_student['reading_score'] >= 70,'student_name'].count()/total_students)*100,1)

#print(f"{round(prcnt_pass_read,3)*100}%")


In [128]:
#Percent passing reading
prcnt_pass_math = round((load_student.loc[load_student['math_score'] >= 70,'student_name'].count()/total_students)*100,1)

#print(f"{round(prcnt_pass_math,3)*100}%")

In [133]:
#Overall pass rate
overall = round(((prcnt_pass_read + prcnt_pass_math)/2),1)
print(f"{overall}%")

80.4%


In [164]:
#District Summary Table
district_sum_headers = ['Total Schools','Total Students', 'Total Budget', 'Average Math Score','Average Reading Score','% Passing Math', '% Passing Reading','Overall Pass Rate']
district_data = [total_schools,total_students,total_budget,avg_math,avg_reading,prcnt_pass_math,prcnt_pass_read,overall]
district_summary = pd.DataFrame({'Metric':district_sum_headers,'Result':district_data})
district_summary.set_index('Metric',inplace=True)
district_summary_tran = district_summary.T



In [165]:

district_summary_tran[['Total Schools','Total Students']] = district_summary_tran[['Total Schools','Total Students']].astype(int)


district_summary_tran['Overall Pass Rate'] = pd.Series(["{0:.1f}%".format(val) for val in district_summary_tran['Overall Pass Rate']], index = district_summary_tran.index)
district_summary_tran['Total Budget'] = pd.Series(["${0:.2f}".format(val) for val in district_summary_tran['Total Budget']], index = district_summary_tran.index)
district_summary_tran

Metric,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate
Result,15,39170,$24649428.00,79.00,81.90,75.00,85.80,80.4%


In [75]:
#Merge two data sets based on school_name column
all_data = load_student.merge(load_school,on='school_name',how='inner')
all_data.drop(['School ID','Student ID'],axis=1, inplace=True)
all_data.columns = ['Student Name','Gender','Grade','School Name','Reading Score','Math Score','Type','Size','Budget']
all_data.head()

,Student Name,Gender,Grade,School Name,Reading Score,Math Score,Type,Size,Budget
0,Paul Bradley,M,9th,Huang High School,66,79,District,2917,1910635
1,Victor Smith,M,12th,Huang High School,94,61,District,2917,1910635
2,Kevin Rodriguez,M,12th,Huang High School,90,60,District,2917,1910635
3,Dr. Richard Scott,M,12th,Huang High School,67,58,District,2917,1910635
4,Bonnie Ray,F,9th,Huang High School,97,84,District,2917,1910635


In [76]:
#Generate dataframe of mean scores
school_sum_df = all_data.groupby('School Name').mean()
school_sum_df['$ Per Student'] = school_sum_df['Budget']/school_sum_df['Size']
#school_sum_df.rename_axis('School Name',inplace=True)
#school_sum_df.index.name = 'School Name'
school_sum_df.head()

,Reading Score,Math Score,Size,Budget,$ Per Student
School Name,,,,,
Bailey High School,81.033963,77.048432,4976.0,3124928.0,628.0
Cabrera High School,83.975780,83.061895,1858.0,1081356.0,582.0
Figueroa High School,81.158020,76.711767,2949.0,1884411.0,639.0
Ford High School,80.746258,77.102592,2739.0,1763916.0,644.0
Griffin High School,83.816757,83.351499,1468.0,917500.0,625.0


In [77]:
#Get percent passing reading and math

pass_df = all_data.groupby('School Name')[['Reading Score','Math Score']].apply(lambda x: (x >= 70).sum()).reset_index()
#df11=df.groupby('key1')['key2'].apply(lambda x: (x=='one').sum()).reset_index(name='count')
pass_df = pass_df.set_index(['School Name'])
pass_df['% Passing Math'] = round((pass_df['Math Score']/school_sum_df['Size'])*100,1)
pass_df['% Passing Reading'] = round((pass_df['Reading Score']/school_sum_df['Size'])*100,1)   
pass_df

,Reading Score,Math Score,% Passing Math,% Passing Reading
School Name,,,,
Bailey High School,4077,3318,66.7,81.9
Cabrera High School,1803,1749,94.1,97.0
Figueroa High School,2381,1946,66.0,80.7
Ford High School,2172,1871,68.3,79.3
Griffin High School,1426,1371,93.4,97.1
Hernandez High School,3748,3094,66.8,80.9
Holden High School,411,395,92.5,96.3
Huang High School,2372,1916,65.7,81.3
Johnson High School,3867,3145,66.1,81.2


In [51]:
#Select columns and create new df with only % passing columns
sub_prcnt_df = pass_df[['% Passing Math','% Passing Reading']]
#Add % pass columns to schools summary
final_school_df = pd.concat([school_sum_df,sub_prcnt_df],axis=1,sort=True)


In [78]:
#Add overall passing based on % math and % read passing
final_school_df['Overall Passing'] = (final_school_df['% Passing Math'] + final_school_df['% Passing Reading'])/2

final_school_df

,Reading Score,Math Score,Size,Budget,$ Per Student,% Passing Math,% Passing Reading,Overall Passing
school_name,,,,,,,,
Bailey High School,81.033963,77.048432,4976.0,3124928.0,628.0,66.7,81.9,74.30
Cabrera High School,83.975780,83.061895,1858.0,1081356.0,582.0,94.1,97.0,95.55
Figueroa High School,81.158020,76.711767,2949.0,1884411.0,639.0,66.0,80.7,73.35
Ford High School,80.746258,77.102592,2739.0,1763916.0,644.0,68.3,79.3,73.80
Griffin High School,83.816757,83.351499,1468.0,917500.0,625.0,93.4,97.1,95.25
Hernandez High School,80.934412,77.289752,4635.0,3022020.0,652.0,66.8,80.9,73.85
Holden High School,83.814988,83.803279,427.0,248087.0,581.0,92.5,96.3,94.40
Huang High School,81.182722,76.629414,2917.0,1910635.0,655.0,65.7,81.3,73.50
Johnson High School,80.966394,77.072464,4761.0,3094650.0,650.0,66.1,81.2,73.65


In [79]:
#Show top five schools sorted by overall passing percentage
top_five = final_school_df.sort_values('Overall Passing',ascending=False).head()
top_five

,Reading Score,Math Score,Size,Budget,$ Per Student,% Passing Math,% Passing Reading,Overall Passing
school_name,,,,,,,,
Cabrera High School,83.975780,83.061895,1858.0,1081356.0,582.0,94.1,97.0,95.55
Thomas High School,83.848930,83.418349,1635.0,1043130.0,638.0,93.3,97.3,95.30
Griffin High School,83.816757,83.351499,1468.0,917500.0,625.0,93.4,97.1,95.25
Pena High School,84.044699,83.839917,962.0,585858.0,609.0,94.6,95.9,95.25
Wilson High School,83.989488,83.274201,2283.0,1319574.0,578.0,93.9,96.5,95.20


In [80]:
#Show bottom five schools sorted by overall passing percentage
bottom_five = final_school_df.sort_values('Overall Passing',ascending=False).tail()
bottom_five

,Reading Score,Math Score,Size,Budget,$ Per Student,% Passing Math,% Passing Reading,Overall Passing
school_name,,,,,,,,
Ford High School,80.746258,77.102592,2739.0,1763916.0,644.0,68.3,79.3,73.80
Johnson High School,80.966394,77.072464,4761.0,3094650.0,650.0,66.1,81.2,73.65
Huang High School,81.182722,76.629414,2917.0,1910635.0,655.0,65.7,81.3,73.50
Figueroa High School,81.158020,76.711767,2949.0,1884411.0,639.0,66.0,80.7,73.35
Rodriguez High School,80.744686,76.842711,3999.0,2547363.0,637.0,66.4,80.2,73.30


In [87]:
#Table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
#Replace the strings for grade level with integers so they can be sorted automatically
mean_by_grade = all_data.replace(['9th','10th','11th','12th'],[9,10,11,12])
#Group by school and grade and calculate mean math and reading score
mean_grade_summary = pd.pivot_table(mean_by_grade, values=['Reading Score','Math Score'], index=['School Name','Grade'], aggfunc=np.mean)
#mean_grade_summary['Math Score','Reading Score'].astype('int')
mean_grade_summary.head()



Math Score  Reading Score
School Name         Grade                           
Bailey High School  9       77.083676      81.303155
                    10      76.996772      80.907183
                    11      77.515588      80.945643
                    12      76.492218      80.912451
Cabrera High School 9       83.094697      83.676136

In [82]:
#Summary statistics to help with deriving bins
final_school_df['$ Per Student'].describe()

count     15.000000
mean     620.066667
std       28.544368
min      578.000000
25%      591.500000
50%      628.000000
75%      641.500000
max      655.000000
Name: $ Per Student, dtype: float64

In [83]:
bins = [0,598,618,638,658]
spend_labels = ['Low','Low-Mid','Mid','High']
#df["Test Score Summary"] = pd.cut(df["Test Score"], bins, labels=group_names)
final_school_df['Spend Category'] = pd.cut(final_school_df['$ Per Student'],bins,labels=spend_labels)
by_spend_df = final_school_df.groupby(['Spend Category']).mean()

#final_headers = ['Reading Score', 'Math Score', 'Size', 'Budget', '$ Per Student', '% Passing Math', '% Passing Reading', 'Overall Passing']
#by_spend_df.columns = final_headers
by_spend_df

,Reading Score,Math Score,Size,Budget,$ Per Student,% Passing Math,% Passing Reading,Overall Passing
Spend Category,,,,,,,,
Low,83.933814,83.455399,1592.0,924604.25,581.0,93.45,96.600,95.0250
Low-Mid,83.885211,83.599686,1361.5,821229.00,604.5,94.25,95.900,95.0750
Mid,82.361084,80.165248,3019.5,1908230.25,632.0,79.95,89.125,84.5375
High,80.997561,76.961198,3600.2,2335126.40,648.0,66.58,80.680,73.6300


In [ ]:
#Get sum stats of Size to use for bins
final_school_df['Size'].describe()

In [84]:
#Repeat the above breakdown, but this time group schools based on a reasonable approximation of school size (Small, Medium, Large).
size_bins = [0,1700,3400,5100]
size_labels = ['Small','Medium','Large']
final_school_df['Size Category'] = pd.cut(final_school_df['Size'],size_bins,labels=size_labels)
by_size_df = final_school_df.groupby(['Size Category']).mean()
by_size_df.columns = final_headers
by_size_df

,Reading Score,Math Score,Size,Budget,$ Per Student,% Passing Math,% Passing Reading,Overall Passing
Size Category,,,,,,,,
Small,83.881343,83.603261,1123.000000,6.986438e+05,613.250000,93.450000,96.650000,95.050000
Medium,82.676142,80.545935,2329.571429,1.437985e+06,611.571429,82.171429,89.614286,85.892857
Large,80.919864,77.063340,4592.750000,2.947240e+06,641.750000,66.500000,81.050000,73.775000


In [85]:
#Repeat the above breakdown, but this time group schools based on school type (Charter vs. District).
#Generate df of unique schools and the type
unique_schools_type = all_data.drop_duplicates(subset='School Name',keep='first').set_index('School Name').sort_values(['School Name'])['Type']
#Concat the unique df with existing school summary df
final_school_df['Type'] = unique_schools_type
#Group by type and calculate means
group_by_type = final_school_df.groupby(['Type']).mean()

group_by_type

,Reading Score,Math Score,Size,Budget,$ Per Student,% Passing Math,% Passing Reading,Overall Passing
Type,,,,,,,,
Charter,83.896421,83.473852,1524.250000,9.126881e+05,599.500000,93.625000,96.575000,95.100000
District,80.966636,76.956733,3853.714286,2.478275e+06,643.571429,66.571429,80.785714,73.678571
